In [1]:
%pylab widget

Populating the interactive namespace from numpy and matplotlib


In [42]:
import os
import sys
import tqdm
import multiprocessing as mp
import multiprocessing.pool as mp_pool
import optimizers as opt
import uuid

In [3]:
def Heaviside(val, cond):
    if cond:
        return val
    else:
        return 0

In [146]:
Num_Days = 16384
Num_Days_Input = 1
Num_Features = 10
Num_Output_Features = 1

train_test_ratio = 0.75
val_test_ratio = 0.125

np.random.seed(1)
dataX, dataY = np.random.rand(*(Num_Days, Num_Features)), np.random.rand(*(Num_Days, Num_Output_Features))

In [153]:
class NNIndividual(opt.Individual):
    parameter_names = ['optimizer', 'loss', 'dense_input_units_count', 'dense_activation', 
                       'l2_reg_parameter', 'l1_reg_parameter',
                       'training_epochs', 'batch_size']
    
    def __init__(self, optimizer, loss, dense_input_units_count, dense_activation, l2_reg_parameter, l1_reg_parameter, 
                 training_epochs, batch_size):
        self.parameters = {'optimizer': optimizer, 'loss': loss, 
                           'dense_input_units_count': dense_input_units_count,
                           'dense_activation': dense_activation,
                           'l2_reg_parameter': l2_reg_parameter,
                           'l1_reg_parameter': l1_reg_parameter,
                           'training_epochs': training_epochs,
                           'batch_size': batch_size}
        self.is_fitness_computed = False
        self.model_dir = os.path.join("temp/", str(uuid.uuid1())+"/")
        os.mkdir(self.model_dir)
        self.model_name = os.path.join(self.model_dir, "model.{epoch:02d}-{val_loss:.2f}.hdf5")
        
    def mp_model_and_fit_process(self, model_name, model_dir, dataX, dataY, kwargs):
        import sys
        import os
        stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')
        stderr = sys.stderr
        sys.stderr = open(os.devnull, 'w')
        try:
            import warnings 
            with warnings.catch_warnings(): 
                warnings.filterwarnings("ignore",category=FutureWarning)
                import keras
                from keras.preprocessing.sequence import TimeseriesGenerator
                sys.stdout = stdout
                sys.stderr = stderr
                import keras.backend as K
                import tensorflow as tf
                import gc
            
            end_index = int(train_test_ratio*dataX.shape[0])
            end_index -= end_index % kwargs['batch_size']
            if end_index <= 0:
                end_index = kwargs['batch_size']
                
            end_index_val = int(val_test_ratio*(dataX.shape[0] - end_index))
            end_index_val -= end_index_val % kwargs['batch_size']
            if end_index_val <= 0:
                end_index_val = kwargs['batch_size']
                
            train_gen = TimeseriesGenerator(dataX, dataY, 
                                            length=Num_Days_Input,
                                            sampling_rate=1,
                                            stride=1,
                                            batch_size=kwargs['batch_size'],
                                            end_index=end_index)
            
            val_gen = TimeseriesGenerator(dataX, dataY,
                                          length=Num_Days_Input,
                                          sampling_rate=1,
                                          stride=1,
                                          batch_size=kwargs['batch_size'],
                                          start_index=end_index+1,
                                          end_index=end_index_val + end_index)
            
            test_gen = TimeseriesGenerator(dataX, dataY, 
                                           length=Num_Days_Input,
                                           sampling_rate=1,
                                           stride=1,
                                           batch_size=kwargs['batch_size'],
                                           start_index=end_index_val + end_index,
                                           end_index=dataX.shape[0]-1)
            
            input_layer = keras.layers.Input(shape=train_gen[0][0].shape[1:])
            dense_layer = keras.layers.Dense(kwargs['dense_input_units_count'], activation=kwargs['dense_activation'],
                                              kernel_regularizer=keras.regularizers.l2(kwargs['l2_reg_parameter']),
                                              activity_regularizer=keras.regularizers.l1(kwargs['l1_reg_parameter']))(input_layer)

            reshape_layer = keras.layers.LSTM(kwargs['dense_input_units_count'], activation=kwargs['dense_activation'],
                                              kernel_regularizer=keras.regularizers.l2(kwargs['l2_reg_parameter']),
                                              activity_regularizer=keras.regularizers.l1(kwargs['l1_reg_parameter']))(dense_layer)
            
            output_layer = keras.layers.Dense(train_gen[0][1].shape[-1], activation="linear")(reshape_layer)
            model = keras.models.Model(inputs=input_layer, outputs=output_layer)
            model.compile(loss=kwargs['loss'], optimizer=kwargs['optimizer'])
            
#             model.summary()
            test_csv_cb = keras.callbacks.CSVLogger(os.path.join(model_dir, 'progress.csv'), separator=',', append=False)
            early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, monitor='val_loss')
            model_saver_cb = keras.callbacks.ModelCheckpoint(model_name, monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)
            model.fit_generator(train_gen, validation_data=val_gen, epochs=kwargs['training_epochs'], verbose=0, 
                                callbacks=[early_stopping_cb, model_saver_cb,test_csv_cb], workers=1, use_multiprocessing=False)
            model.save(model_name)

            gc.collect()
     
            print(model.predict_generator(test_gen).shape)

            return model.evaluate_generator(test_gen)
        except Exception as e:
            sys.stdout = stdout
            sys.stderr = stderr
            raise e      
    
    def mp_model_and_predict(self, model_name, test_values):
        try: 
            import warnings 
            with warnings.catch_warnings(): 
                warnings.filterwarnings("ignore",category=FutureWarning)
                import keras
                import keras.backend as K
                import gc
            model = keras.models.load_model(model_name)

            gc.collect()

            return model.predict(test_values)
        except Exception as e:
            raise e 
    
    def get_fitness(self):
        if not self.is_fitness_computed:
            try:
                with mp.Pool(processes=1) as pool:
                    self.fitness_value = pool.apply(self.mp_model_and_fit_process, (self.model_name, self.model_dir, dataX, dataY, self.parameters))
            except:
                raise
            else:
                self.is_fitness_computed = True
        return self.fitness_value
    
    def predict(self, test_values):
        if not self.is_fitness_computed:
            try:
                self.get_fitness()
            except:
                raise
        try:
            with mp.Pool(processes=1) as pool:
                predictions = pool.apply(self.mp_model_and_predict, (self.model_name, test_values))
        except:
            raise
        else:
            return predictions

In [154]:
parameter_space = {'optimizer': opt.RandomGenerator(str, ['rmsprop', 'adam'], True), 
                   'loss': opt.RandomGenerator(str, ['mse'], True), 
                   'dense_input_units_count': opt.RandomGenerator(int, [i*2 + 32 for i in range(17)]),
                   'dense_activation': opt.RandomGenerator(str, ['selu', 'linear', 'relu', 'tanh'], True),
                   'l2_reg_parameter': opt.RandomGenerator(float, [1e-4, 1e-3], False),
                   'l1_reg_parameter': opt.RandomGenerator(float, [1e-5, 1e-4], False),
                   'training_epochs': opt.RandomGenerator(int, list(range(10, 200, 10)), True),
                   'batch_size': opt.RandomGenerator(int, [128, 256, 512, 1024, 2048], True)
                  }

NNOptimizer = opt.GAOptimizer(NNIndividual, 10, parameter_space, 3, 0.1)

In [155]:
with mp_pool.ThreadPool(processes=12) as pool:
    NNOptimizer.evolve_multigeneration(10, verbose=0)

for i in TemplateOptimizer[0:5]:
    print(str(i))

(3071, 1)
(3583, 1)



KeyboardInterrupt: 